# Data Cleaning

---

Notebook ini digunakan untuk proses cleaning data hasil scraping

Proses cleaning dilakukan untuk membersihkan data hasil scraping, agar data lebih relable untuk digunkan untuk proses selanjutnya (analysis dan model).

---

# I. Import Libaries

In [1]:
# Import Libraries

import json
import pandas as pd
import numpy as np
import re
import ast
from langdetect import detect, DetectorFactory
import warnings
warnings.filterwarnings('ignore')


# II. Load Data

Data file json hasil scraping di load dan digabungkan, kemudian disave sebagai backup.

In [2]:
# Read file JSON
with open('data-scrap/amazfit.json') as f1, open('data-scrap/apple.json') as f2, open('data-scrap/fitbit.json') as f3, open('data-scrap/garmin.json') as f4, open('data-scrap/google.json') as f5, open('data-scrap/huawei.json') as f6, open('data-scrap/samsung.json') as f7, open('data-scrap/xiaomi.json') as f8:
    data1 = json.load(f1)
    data2 = json.load(f2)
    data3 = json.load(f3)
    data4 = json.load(f4)
    data5 = json.load(f5)
    data6 = json.load(f6)
    data7 = json.load(f7)
    data8 = json.load(f8)

# Combine
combine = data1 + data2 + data3 + data4 + data5 + data6 + data7 + data8

# Save as backup
with open('all.json', 'w') as outfile:
    json.dump(combine, outfile, indent=2, ensure_ascii=True)

In [3]:
# Read file JSON
with open('all.json', 'r') as file:
    data = json.load(file)

# To DataFrame
df = pd.DataFrame(data)

# Show DataFrame
df


,title,rating,price,features,Battery Capacity,Connectivity,GPS,Screen Size,img_url,Brand,Review
0,"Amazfit Bip 6 Smart Watch 46mm, 14 Day Battery...",4.5 out of 5 stars,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
...,...,...,...,...,...,...,...,...,...,...,...
537,Xiaomi Smart Band 9 Global Version (2024) 1.62...,4.5 out of 5 stars,$57.99,Heart Rate Monitor,233 Milliamp Hours,Bluetooth,No GPS,1.62 Inches,https://m.media-amazon.com/images/I/61WgY4Yssc...,Xiaomi,[{'review': 'This is the 4th generation of Mi ...
538,"Xiaomi Redmi Watch 5 Smartwatch, 2.07"" LCD Dis...",4.4 out of 5 stars,$139.99,"Voice Call, Lightweight, Activity Tracker, Pho...",N/A,Bluetooth,No GPS,2.07 Inches,https://m.media-amazon.com/images/I/51X9nE628c...,Xiaomi,[{'review': 'Calidad - precio. El producto es ...
539,"Xiaomi Watch 2 Pro, Google apps Directly on Yo...",4.1 out of 5 stars,$259.99,Bluetooth,N/A,Bluetooth,Built-in GPS,1.43 Inches,https://m.media-amazon.com/images/I/81LQkI4eoq...,Xiaomi,"[{'review': 'Me encanta este reloj, cumple con..."
540,"Xiaomi Redmi 5 Lite Smart Watch, Step Counter ...",4.6 out of 5 stars,$52.24,"Multisport Tracker, Activity Tracker, Alarm Cl...",470 Amp Hours,Bluetooth,N/A,2 Inches,https://m.media-amazon.com/images/I/61Mr-G86wZ...,Xiaomi,[{'review': 'Although positioned as an entry-l...


Baca file dengan pandas, kemudian save sebagai csv sebgai backup.

In [4]:
# Save to CSV as backup
df.to_csv('data_before_clean.csv', index=False)

In [5]:
# Data Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             542 non-null    object
 1   rating            542 non-null    object
 2   price             542 non-null    object
 3   features          542 non-null    object
 4   Battery Capacity  542 non-null    object
 5   Connectivity      542 non-null    object
 6   GPS               542 non-null    object
 7   Screen Size       542 non-null    object
 8   img_url           542 non-null    object
 9   Brand             542 non-null    object
 10  Review            542 non-null    object
dtypes: object(11)
memory usage: 46.7+ KB


Nama kolom diubah agar lebih mudah.

In [6]:
# Change Column Name
df.rename(columns={
    'title': 'products',
    'Battery Capacity': 'battery',
    'Connectivity': 'connectivity',
    'GPS': 'gps',
    'Screen Size': 'screen_size',
    'Brand': 'brand',
    'Review': 'reviews'
}, inplace=True)

df.head()


,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,"Amazfit Bip 6 Smart Watch 46mm, 14 Day Battery...",4.5 out of 5 stars,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2 Sport Smart Watch Fitness Tra...,4.5 out of 5 stars,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3 Outdoor Smart Watch 48mm AMOLE...,4.4 out of 5 stars,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


Nama kolom sudah terubah.

Memastikan brand yang diinginkan ada pada dataset.

In [7]:
# Check brand
df.brand.unique()

array(['Amazfit', 'Apple Watch', 'Fitbit', 'Garmin', 'Google', 'Huawei',
       'Samsung', 'Xiaomi'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   products      542 non-null    object
 1   rating        542 non-null    object
 2   price         542 non-null    object
 3   features      542 non-null    object
 4   battery       542 non-null    object
 5   connectivity  542 non-null    object
 6   gps           542 non-null    object
 7   screen_size   542 non-null    object
 8   img_url       542 non-null    object
 9   brand         542 non-null    object
 10  reviews       542 non-null    object
dtypes: object(11)
memory usage: 46.7+ KB


# III. Data Cleaning

### 1. Column product

In [9]:
# Change 'N/A' to NaN
df['products'] = df['products'].replace('N/A', np.nan)

In [10]:
# Drop Data null in products
df = df.dropna(subset=['products']).reset_index(drop=True)

Drop missing value jika ada.

In [11]:
# Filter products based on Brand

# Create regex pattern with keyword
pattern = r'^(Amazfit|Apple Watch|Fitbit|Garmin|Xiaomi|Huawei Watch|Samsung|Google Pixel)'

# Filter products with keyword product (case-insensitive)
df = df[df['products'].str.contains(pattern, case=False, na=False, regex=True)]

# Reset index
df = df.reset_index(drop=True)


Filtering product yang hanya mencantum brand smart watch yang ditentukan.

In [12]:
# Make sure brand is in product name
df = df[df.apply(lambda row: str(row['brand']).lower() in str(row['products']).lower(), axis=1)]

Memastikan data di brand tercantum di kolom nama produk (products)

In [13]:
# Delete data that have accesorris in products
df = df[~df['products'].str.contains(r'\b(cable|carger)\b', case=False, na=False)]

Menghapus data yang merupkan aksesoris dari smartwatch, bukan smart watch itu sendiri.

In [14]:
# Strip products name after delimiter
def cut_at_first_delimiter(text):
    if not isinstance(text, str):
        return text
    
    delimiters = [',', '，', '(', '[', ' -']
    positions = [text.find(d) for d in delimiters if text.find(d) != -1]
    
    if positions:
        cut_pos = min(positions)
        return text[:cut_pos].strip()
    else:
        return text.strip()

df['products'] = df['products'].apply(cut_at_first_delimiter)


Memotong nama produk sesuai tanda baca, yang biasanya setelahnya merupkan keterangan produk.

In [15]:
# Strip products name after number
def cut_after_last_number(text):
    if not isinstance(text, str):
        return text
    
    match = re.search(r'^(.*?\d+)(?:\s|$)', text)
    
    if match:
        return match.group(1).strip()
    else:
        return text.strip()
    
df['products'] = df['products'].apply(cut_after_last_number)

Memotong nama produk setekah angka, angka biasanya no series produk, setelahnya biasanya merrupkan keterangan produk.

In [16]:
# Strip amount of words
def extract_product_name(text):
    if not isinstance(text, str):
        return text
    
    # Take only 5 words
    return ' '.join(text.split()[:5])

df['products'] = df['products'].apply(extract_product_name)


Membatasi nama produk hanya dalam 5 kata.

In [17]:
# Remove uncessery word from 'products'
df['products'] = df['products'].str.replace(r'with|by', '', regex=True).str.strip()

Menghapus kata-kata seperti with dan by.

In [18]:
# Remove symbol from 'products'
df['products'] = df['products'].str.replace(r'[®™]', '', regex=True).str.strip()

Menghapus symbol-symbol yang berada pada nama produk.

In [19]:
df.products.unique()

array(['Amazfit Bip 6', 'Amazfit Active 2', 'Amazfit T-Rex 3',
       'Amazfit GTR Mini Smart Watch', 'Amazfit Band 7',
       'Amazfit Active Smart Watch 42mm',
       'Amazfit Balance Smart Watch 46mm', 'Amazfit Refurbished BIP 3',
       'Amazfit Bip 5', 'Amazfit GTR 3', 'Amazfit Balance 2',
       'Amazfit Refurbished T-Rex 3', 'Amazfit Renewed Bip 5',
       'Amazfit Bip 3', 'Amazfit GTS 2',
       'Amazfit T-Rex Ultra Tactical Smart', 'Apple Watch Series 10',
       'Apple Watch Ultra 2', 'Apple Watch Series 5',
       'Apple Watch Series 7', 'Apple Watch Series 6',
       'Apple Watch Series 8', 'Apple Watch SE', 'Apple Watch Ultra',
       'Apple Watch SE 2nd Gen', 'Apple Watch Series 9',
       'Apple Watch Series SE 44mm', 'Fitbit Versa 4', 'Fitbit Charge 6',
       'Fitbit Inspire 3', 'Fitbit Sense 2', 'Fitbit Versa 2',
       'Fitbit Google Ace LTE', 'Fitbit Versa Smart Watch',
       'Fitbit Versa Lite Smartwatch', 'Fitbit Sense Advanced Smartwatch',
       'Google Pixel W

Kolom nama produk sudah lebih pendek dan sesuai.

In [20]:
# Checking product weather is smartwatch or not

# Group brand -> unique products
unique_products_df = (
    df.groupby('brand')['products']
      .apply(lambda x: ', '.join(sorted(x.unique())))
      .reset_index()
      .rename(columns={'products': 'unique_products'})
)

print(unique_products_df.to_string(index=False))


      brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Dari hasil akan dicek kembali jika ada data yang bukan smartwatch. Ini diperlukan pengecekan produk satu per satu. Tidak bisa otomatis.

In [21]:
# List product non-smartwatch
non_smartwatch = [
    'Garmin Dash Cam Mini 3', 'Garmin Drive 52', 'Garmin Drive 53', 'Garmin DriveSmart 55',
    'Garmin DriveSmart 66', 'Garmin DriveSmart 86', 'Garmin RV 1095', 'Garmin dēzl OTR1000',
    'Garmin eLog', 'Garmin Tread Powersport Off-Road Navigator', 'Garmin Striker Vivid 5cv',
    'Garmin Approach R10', 'Garmin Approach Z30', 'Garmin Approach Z82', 'Garmin ECHOMAP UHD2',
    'Garmin Xero C1', 'Garmin Suction cup mount', 'Garmin inReach Messenger Handheld Satellite',
    'Garmin 010-02060-00', 'Garmin 010-02376-00', 'Garmin 010-02430-00', 'Garmin 010-02638-10', 'Garmin 010-12883-00',
    'Garmin Edge 1040', 'Garmin Edge 840', 'Garmin GPSMAP 65s', 'Garmin GPSMAP 67i Rugged GPS', 'Garmin Index S2', 'Garmin zūmo XT2'
]

# Drop datas
df = df[~df['products'].isin(non_smartwatch)].reset_index(drop=True)

In [22]:
# Change format case and delete space
df['products'] = df['products'].str.strip().str.title()

In [23]:
# Checking product

# Group brand -> unique products
unique_products_df = (
    df.groupby('brand')['products']
      .apply(lambda x: ', '.join(sorted(x.unique())))
      .reset_index()
      .rename(columns={'products': 'unique_products'})
)

# Tampilkan semua baris
print(unique_products_df.to_string(index=False))

      brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                           unique_products
    Amazfit                                                                                                                       Amazfit Active 2, Amazfit Active Smart Watch 42Mm, Amazfit Balance 2, Amazfit Balance Smart Watch 46Mm, Amazfit Band 7, Amazfit Bip 3, Amazfit Bip 5, Amazfit Bip 6, Amazfit Gtr 3, Amazfit Gtr Mini Smart Watch, Amazfit Gts 2, Amazfit Refurbished Bip 3, Amazfit Refurbished T-Rex 3, Amazfit Renewed Bip 5, Amazfit T-Rex 3, Amazfit T-Rex Ultra Tactical Smart
Apple Watch                 

Sudah tidak ada data produk yang bukan smartwatch

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   products      373 non-null    object
 1   rating        373 non-null    object
 2   price         373 non-null    object
 3   features      373 non-null    object
 4   battery       373 non-null    object
 5   connectivity  373 non-null    object
 6   gps           373 non-null    object
 7   screen_size   373 non-null    object
 8   img_url       373 non-null    object
 9   brand         373 non-null    object
 10  reviews       373 non-null    object
dtypes: object(11)
memory usage: 32.2+ KB


### 2. Column rating

Kolom rating perlu dilakukan striping setelah angka rating dan hasil diubah menjadi numerikal.

In [25]:
# Clean and Convert rating
def cut_after_space_and_convert(text):
    if not isinstance(text, str):
        return np.nan

    first_part = text.split(' ')[0]
    try:
        return float(first_part)
    except ValueError:
        return np.nan

df['rating'] = df['rating'].apply(cut_after_space_and_convert)

In [26]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,$74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,$75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,$189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         373 non-null    object 
 3   features      373 non-null    object 
 4   battery       373 non-null    object 
 5   connectivity  373 non-null    object 
 6   gps           373 non-null    object 
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(1), object(10)
memory usage: 32.2+ KB


Kolom rating sudah bersih dan berbentuk numerical.

### 3. Column price

Kolom harga akan di strip bagian non numeric nya sepertii tanda dollar dan diubah ke bentuk numerical.

In [28]:
# Change 'N/A' to NaN
df['price'] = df['price'].replace('N/A', np.nan)

In [29]:
# Clean and convert price
df['price'] = df['price'].astype(str).str.replace(r'[^\d\.]', '', regex=True)

# Convert to float
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [30]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340 Milliamp Hours,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270 Milliamp Hours,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",N/A,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      373 non-null    object 
 4   battery       373 non-null    object 
 5   connectivity  373 non-null    object 
 6   gps           373 non-null    object 
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(2), object(9)
memory usage: 32.2+ KB


Kolom price sudah bersih dan berbentuk type numerical.

### 4. Column features

Kolom features tidak ada perubahan, hanya kalau memiliki 'N/A' diubah ke null.

In [32]:
# Change 'N/A' to NaN
df['features'] = df['features'].replace('N/A', np.nan)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       373 non-null    object 
 5   connectivity  373 non-null    object 
 6   gps           373 non-null    object 
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(2), object(9)
memory usage: 32.2+ KB


### 5. Column battery

Kolom battery dilakukan pemotongan non numeric karakter dan konversi semua ke mah. Kemudian convert ke numerical type.

In [34]:
# Clean and Change battery

def clean_battery(value):
    if not isinstance(value, str):
        return np.nan

    value = value.strip().lower()

    if value in ['n/a', 'na']:
        return np.nan

    # Handle scientific notation (e.g., 1E+1)
    if re.match(r'^\d+(\.\d+)?[eE][+-]?\d+', value):
        try:
            return float(value)
        except:
            return np.nan

    # Extract numeric value
    match = re.search(r'(\d+(\.\d+)?([eE][+-]?\d+)?)', value)
    if not match:
        return np.nan

    num = float(match.group(1))

    # Convert based on unit
    if 'milliamp' in value or 'mah' in value:
        return num
    elif re.search(r'\bamp(?:ere)? hour', value):  # e.g., 0.3 amp hour
        return num * 1000
    elif 'day' in value:
        return num * 24 * 25  # 25mA per hour × 24 jam, 25mA average current for smartwatch
    elif 'hour' in value or 'hr' in value:
        return num * 25       # 25mA per hour
    elif re.fullmatch(r'\d+(\.\d+)?', value):  # plain number
        return num
    else:
        return np.nan
    
df['battery'] = df['battery'].apply(clean_battery)

In [35]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,Built-in GPS,1.97 Inches,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,Built-in GPS,1.32 Inches,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",Built-in GPS,1.5 Inches,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  373 non-null    object 
 6   gps           373 non-null    object 
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(3), object(8)
memory usage: 32.2+ KB


Kolom battery sudah bersih dan dikonversi ke mAh. Juga sudah berbentuk numerical.

### 6. Column connectivity

Kolom connectivity tidak ada perubahan, hanya kalau memiliki 'N/A' diubah ke null.

In [37]:
# Change 'N/A' to NaN
df['connectivity'] = df['connectivity'].replace('N/A', np.nan)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  358 non-null    object 
 6   gps           373 non-null    object 
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(3), object(8)
memory usage: 32.2+ KB


### 7. Column gps

Kolom gps memiliki isi string yang bervariasi karena hasil scraping, kolom gps berfungsi untuk mengetahui apakah gps tersedia atau tidak pada produk.

In [39]:
# Check unique value
df.gps.unique()

array(['Built-in GPS', 'GPS Enabled', 'GPS Via Smartphone', 'N/A', 'True',
       'built-in gps', 'Yes', 'Gps enabled', 'No GPS',
       'Gps Route Tracking'], dtype=object)

Dilakukan konversi ke nilai 1 dan 0 atau punya dan tidak punya gps. Sehingga memudahkan analysis.

In [40]:
# Cleaning and convert gps column
def clean_gps(value):
    if not isinstance(value, str):
        return np.nan
    
    value = value.lower().strip()
    
    yes_values = [
        'built-in gps', 'gps enabled', 'gps via smartphone',
        'yes', 'true', 'gps route tracking'
    ]
    
    no_values = ['no gps', 'no', 'false']
    
    if value in yes_values:
        return 1
    elif value in no_values:
        return 0
    else:
        return np.nan  # for 'N/A'

df['gps'] = df['gps'].apply(clean_gps)


In [41]:
# Check unique value
df.gps.unique()

array([ 1., nan,  0.])

Hasil sudah 1 and 0 pada kolom gps.

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  358 non-null    object 
 6   gps           352 non-null    float64
 7   screen_size   373 non-null    object 
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(4), object(7)
memory usage: 32.2+ KB


### 8. Column screen_size

Kolom screen_size dilakukan striping satuannya dan kemudian konversi ke inch dan ubah ke numrical type.

In [43]:
# Clean and Convert screen_size
def clean_screen_size(value):

    if not isinstance(value, str):
        return np.nan

    value = value.strip().lower()

    if value in ['N/A']:
        return np.nan

    # Extract numeric value (float or scientific)
    match = re.search(r'(\d+(\.\d+)?([eE][+-]?\d+)?)', value)
    if not match:
        return np.nan

    size = float(match.group(1))

    # Convert based on unit
    if 'millimeter' in value:
        return size / 25.4  # 1 inch = 25.4 mm
    elif 'centimeter' in value:
        return size / 2.54  # 1 inch = 2.54 cm
    else:
        return size  # assume already in inches
    
df['screen_size'] = df['screen_size'].apply(clean_screen_size)

In [44]:
df.head()

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand,reviews
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,1.0,1.97,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,1.0,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,1.0,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",1.0,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",NaN,"Bluetooth, Wi-Fi",1.0,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit,"[{'review': 'So far, after using the watch for..."


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  358 non-null    object 
 6   gps           352 non-null    float64
 7   screen_size   369 non-null    float64
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(5), object(6)
memory usage: 32.2+ KB


Kolom screen_size sudah bersih dan dikonversi ke inch. Juga sudah berbentuk numerical.

### 9. Column img_url

Kolom connectivity tidak ada perubahan, hanya kalau memiliki 'N/A' diubah ke null.

In [46]:
# Change 'N/A' to NaN
df['img_url'] = df['img_url'].replace('N/A', np.nan)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  358 non-null    object 
 6   gps           352 non-null    float64
 7   screen_size   369 non-null    float64
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(5), object(6)
memory usage: 32.2+ KB


# IV. Handling Missing Value

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         372 non-null    float64
 3   features      357 non-null    object 
 4   battery       202 non-null    float64
 5   connectivity  358 non-null    object 
 6   gps           352 non-null    float64
 7   screen_size   369 non-null    float64
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(5), object(6)
memory usage: 32.2+ KB


Dataset merupakan hasil scraping dari website Amazon, hasil scraping kadang mempunyai error dimana tidak berhasilnya data ter scrap atau tidak dicantumkannya secara detail data oleh seller. Pada data set missing value terdapat pada kolom yang merupakan informasi dari produk smartwatch seperti 'battery', 'screen_size', 'features', 'connectivity', 'gps', 'price', karena nilai dari kolom tersebut bergantung pada nama produk pada kasus ini merupakan MAR (Missing at Random). Untuk mengatasi hal ini diperlukan imputasi data untuk missing value berdasarkan referensi untuk mendekati ground truth.

In [49]:
# Handling with if 'products' have same name, the missing value fill in to each other.

# Define column with missing value
cols_to_fill = ['battery', 'screen_size', 'features', 'connectivity', 'gps', 'price']

# Forward and backward fill in group 'products'
for col in cols_to_fill:
    df[col] = df.groupby('products')[col].transform(lambda x: x.ffill().bfill())

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      373 non-null    object 
 1   rating        373 non-null    float64
 2   price         373 non-null    float64
 3   features      361 non-null    object 
 4   battery       290 non-null    float64
 5   connectivity  361 non-null    object 
 6   gps           366 non-null    float64
 7   screen_size   372 non-null    float64
 8   img_url       373 non-null    object 
 9   brand         373 non-null    object 
 10  reviews       373 non-null    object 
dtypes: float64(5), object(6)
memory usage: 32.2+ KB


Dilakukan handling missing value dengan cara mengisi satu sama lain dikolom yang memiliki nama produk sama sebagai referensi.

In [51]:
# Handling missing value by inputing data information from sources about smartwatch

# Load smartwatch specs dataset
df_summary = pd.read_csv('smartwatch_specs_clean.csv')

# Join by 'products' or produk name
df_merged = df.merge(df_summary, on='products', how='left', suffixes=('', '_summary'))

# Define column with missing value
columns_to_fill = ['features', 'battery', 'connectivity', 'gps', 'screen_size']

# Fill in missing value from specs dataset
for col in columns_to_fill:
    df_merged[col] = df_merged[col].fillna(df_merged[f"{col}_summary"])

# Drop column from specs dataset
df_merged.drop(columns=[f"{col}_summary" for col in columns_to_fill], inplace=True)

df = df_merged

# Checking missing value
for col in columns_to_fill:
    print(f"{col} missing: {df[col].isna().sum()} remaining")

features missing: 0 remaining
battery missing: 7 remaining
connectivity missing: 0 remaining
gps missing: 0 remaining
screen_size missing: 0 remaining


Karena data yang mengandung missing value sudah sedikit, akan dilakukan drop agar tidak ada lagi missing value.

In [52]:
# Drop data with missing value
df = df.dropna().reset_index(drop=True)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      377 non-null    object 
 1   rating        377 non-null    float64
 2   price         377 non-null    float64
 3   features      377 non-null    object 
 4   battery       377 non-null    float64
 5   connectivity  377 non-null    object 
 6   gps           377 non-null    float64
 7   screen_size   377 non-null    float64
 8   img_url       377 non-null    object 
 9   brand         377 non-null    object 
 10  reviews       377 non-null    object 
dtypes: float64(5), object(6)
memory usage: 32.5+ KB


Hasil menunjukan sudah tidak ada missing value.

# V. Split Data

In [54]:
# Save before split
df.to_csv('data_before_split.csv', index=False)

Data disave dahulu sebagai backup.

### 1. Data Product

In [55]:
# Data product
df_product = df.drop('reviews', axis=1)
df_product

,products,rating,price,features,battery,connectivity,gps,screen_size,img_url,brand
0,Amazfit Bip 6,4.5,74.99,"Multisport Tracker, Text Messaging, AI Assista...",340.0,Bluetooth,1.0,1.97,https://m.media-amazon.com/images/I/61UvVTN0IE...,Amazfit
1,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,1.0,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit
2,Amazfit Active 2,4.5,75.99,"Multisport Tracker, Text Messaging, AI Assista...",270.0,Bluetooth,1.0,1.32,https://m.media-amazon.com/images/I/71mpuO4Lqe...,Amazfit
3,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",14400.0,"Bluetooth, Wi-Fi",1.0,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit
4,Amazfit T-Rex 3,4.4,189.99,"Maps, Altitude Assistant, Compass, dual-band G...",14400.0,"Bluetooth, Wi-Fi",1.0,1.50,https://m.media-amazon.com/images/I/71GtgMbKvK...,Amazfit
...,...,...,...,...,...,...,...,...,...,...
372,Xiaomi Smart Band 9,4.5,57.99,Heart Rate Monitor,233.0,Bluetooth,0.0,1.62,https://m.media-amazon.com/images/I/61WgY4Yssc...,Xiaomi
373,Xiaomi Redmi Watch 5,4.4,139.99,"Voice Call, Lightweight, Activity Tracker, Pho...",470.0,Bluetooth,0.0,2.07,https://m.media-amazon.com/images/I/51X9nE628c...,Xiaomi
374,Xiaomi Watch 2,4.1,259.99,Bluetooth,7200.0,Bluetooth,1.0,1.43,https://m.media-amazon.com/images/I/81LQkI4eoq...,Xiaomi
375,Xiaomi Redmi 5,4.6,52.24,"Multisport Tracker, Activity Tracker, Alarm Cl...",470000.0,Bluetooth,0.0,2.00,https://m.media-amazon.com/images/I/61Mr-G86wZ...,Xiaomi


In [56]:
# Check duplicate
df_product.duplicated().sum()

97

In [57]:
# Delete duplicate
df_product = df_product.drop_duplicates()

In [58]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 280 entries, 0 to 376
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   products      280 non-null    object 
 1   rating        280 non-null    float64
 2   price         280 non-null    float64
 3   features      280 non-null    object 
 4   battery       280 non-null    float64
 5   connectivity  280 non-null    object 
 6   gps           280 non-null    float64
 7   screen_size   280 non-null    float64
 8   img_url       280 non-null    object 
 9   brand         280 non-null    object 
dtypes: float64(5), object(5)
memory usage: 24.1+ KB


In [59]:
# Save data_product
df_product.to_csv('data_product.csv', index=False)

file csv data product berhasil disave.

### 2. Data Review

In [60]:
# Data review
df_review = df[['products', 'reviews']]
df_review

,products,reviews
0,Amazfit Bip 6,[{'review': 'I purchased this watch to replace...
1,Amazfit Active 2,[{'review': 'This thing blows away the Fitbit ...
2,Amazfit Active 2,[{'review': 'This thing blows away the Fitbit ...
3,Amazfit T-Rex 3,"[{'review': 'So far, after using the watch for..."
4,Amazfit T-Rex 3,"[{'review': 'So far, after using the watch for..."
...,...,...
372,Xiaomi Smart Band 9,[{'review': 'This is the 4th generation of Mi ...
373,Xiaomi Redmi Watch 5,[{'review': 'Calidad - precio. El producto es ...
374,Xiaomi Watch 2,"[{'review': 'Me encanta este reloj, cumple con..."
375,Xiaomi Redmi 5,[{'review': 'Although positioned as an entry-l...


In [61]:
# Split data inside reviews column

# Make sure data is list of dict
def parse_review_column(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else val
    except:
        return np.nan

df_review['reviews'] = df_review['reviews'].apply(parse_review_column)

# Explode
df_exploded = df_review.explode('reviews').reset_index(drop=True)

# Take 'review' dan 'rating' from dict
df_exploded['individual_review'] = df_exploded['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) else np.nan)
df_exploded['individual_rating'] = df_exploded['reviews'].apply(lambda x: x.get('rating') if isinstance(x, dict) else np.nan)

# delete previous reviews column
df_exploded = df_exploded.drop(columns=['reviews'])

# Show result
df_exploded


,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0 out of 5 stars
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0 out of 5 stars
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0 out of 5 stars
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0 out of 5 stars
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0 out of 5 stars
...,...,...,...
2589,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0 out of 5 stars
2590,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0 out of 5 stars
2591,Xiaomi Redmi Watch 4,Me encanta,5.0 out of 5 stars
2592,Xiaomi Redmi Watch 4,,5.0 out of 5 stars


In [62]:
# Clean and Convert individual_rating
def cut_after_space_and_convert(text):
    if not isinstance(text, str):
        return np.nan

    first_part = text.split(' ')[0]  # get substring before first space
    try:
        return float(first_part)
    except ValueError:
        return np.nan

df_exploded['individual_rating'] = df_exploded['individual_rating'].apply(cut_after_space_and_convert)

In [63]:
df_exploded

,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0
...,...,...,...
2589,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0
2590,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0
2591,Xiaomi Redmi Watch 4,Me encanta,5.0
2592,Xiaomi Redmi Watch 4,,5.0


In [64]:
# Keep individual_review in English only
DetectorFactory.seed = 8

def is_english(text):
    if not isinstance(text, str) or text.strip() == '':
        return False
    try:
        return detect(text) == 'en'
    except:
        return False

# apply filter
df_exploded['is_english'] = df_exploded['individual_review'].apply(is_english)

# Drop that is not English
df_reviews_en = df_exploded[df_exploded['is_english']].drop(columns=['is_english']).reset_index(drop=True)

In [65]:
df_reviews_en

,products,individual_review,individual_rating
0,Amazfit Bip 6,I purchased this watch to replace a dead Samsu...,5.0
1,Amazfit Bip 6,"Pleasantly surprised. Finally, a less expensiv...",5.0
2,Amazfit Bip 6,If you’re looking to purchase a smart watch th...,5.0
3,Amazfit Bip 6,"This watch does well, considered the price. Th...",4.0
4,Amazfit Bip 6,This watch is amazing quality for the price! I...,5.0
...,...,...,...
2337,Xiaomi Redmi Watch 4,Great watch for working out and keeping track ...,5.0
2338,Xiaomi Redmi Watch 4,I have had a number of smartwatches including ...,4.0
2339,Xiaomi Redmi Watch 4,Awsome product... Good price and reabilty,5.0
2340,Xiaomi Redmi Watch 4,"My husband used it after he got it, very relia...",5.0


In [66]:
# Check duplicate
df_reviews_en.duplicated().sum()

1239

In [67]:
# Delete duplicate
df_reviews_en = df_reviews_en.drop_duplicates()

In [68]:
# Check if there is 'N/A'
print(df_reviews_en['individual_review'].str.contains('N/A', na=False).any())

False


Tidak ada missing value hasil scraping pada kolom review.

In [69]:
df_reviews_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1103 entries, 0 to 2341
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   products           1103 non-null   object 
 1   individual_review  1103 non-null   object 
 2   individual_rating  1103 non-null   float64
dtypes: float64(1), object(2)
memory usage: 34.5+ KB


In [70]:
# Save data_product
df_reviews_en.to_csv('data_review.csv', index=False)

file csv data review berhasil disave.